In [1]:
from pyspark.sql.functions import current_timestamp

StatementMeta(, 6549076f-cd15-4c90-a714-f0b6b899728c, 3, Finished, Available, Finished)

In [2]:
# Create Config Table in Lakehouse
# This table stores run configuration and status: start_date_time, end_date_time, completion_time, status
config_table_name = "OAP_Load_Config"

spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {config_table_name} (
        start_date_time TIMESTAMP,
        end_date_time TIMESTAMP,
        completion_time TIMESTAMP,
        status STRING
    )
    USING DELTA
""")

print(f"Verified configuration table: {config_table_name}")

StatementMeta(, 6549076f-cd15-4c90-a714-f0b6b899728c, 4, Finished, Available, Finished)

Verified configuration table: OAP_Load_Config


In [3]:
# Create AllowedFabricItems Table in Lakehouse
allowed_items_table_name = "AllowedFabricItems"

spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {allowed_items_table_name} (
        ItemType STRING,
        ModifiedAt TIMESTAMP
    )
    USING DELTA
""")

print(f"Verified table: {allowed_items_table_name}")

StatementMeta(, 6549076f-cd15-4c90-a714-f0b6b899728c, 5, Finished, Available, Finished)

Verified table: AllowedFabricItems


In [4]:
# Define default allowed items
default_items = [
    "VariableLibrary",
    "Semantic Model",
    "Report",
    "App",
    "Dashboard",
    "Scorecard",
    "KQLDashboard",
    "Eventstream",
    "Cosmos DB Database",
    "SQL Database",
    "Databricks Catalog",
    "Azure Data Factory"
]

# Create DataFrame
df_allowed = spark.createDataFrame([(i,) for i in default_items], ["ItemType"])
df_allowed = df_allowed.withColumn("ModifiedAt", current_timestamp())

# Create temporary view for the merge
df_allowed.createOrReplaceTempView("source_allowed_items")

# Merge data: Insert new, Delete missing
spark.sql(f"""
    MERGE INTO {allowed_items_table_name} AS target
    USING source_allowed_items AS source
    ON target.ItemType = source.ItemType
    WHEN NOT MATCHED THEN
        INSERT (ItemType, ModifiedAt) VALUES (source.ItemType, source.ModifiedAt)
    WHEN NOT MATCHED BY SOURCE THEN
        DELETE
""")

print(f"Synced {len(default_items)} types into {allowed_items_table_name}. Removed obsolete items if any.")

StatementMeta(, 6549076f-cd15-4c90-a714-f0b6b899728c, 6, Finished, Available, Finished)

Merged 12 default types into AllowedFabricItems.
